In [13]:
import os
import csv
import numpy as np
from matplotlib import pyplot as plt

In [14]:
def is_number(s):
    """ Returns True if string is a number. """
    try:
        float(s)
        return True
    except ValueError:
        return False

In [18]:
selected_dir = "../benchmarking_results/report_results/timing_benchmarks_heron05"

result_summary_files = []

# Search for the available json files in the results folder:
for subdir, dirs, files in os.walk(selected_dir):
    for file in files:
        if file.endswith(".csv"):
            result_summary_files.append(os.path.join(subdir, file))
    break # <-- This ensures that we only search in a depth of one.
result_summary_files.sort()
for file in result_summary_files:
    print(file.split("/")[-1])


netlib_benchmark_1.0e-4_PDLP+STSN_1_timing.csv
netlib_benchmark_1.0e-4_PDLP+STSN_2_timing.csv
netlib_benchmark_1.0e-4_PDLP+STSN_3_timing.csv
netlib_benchmark_1.0e-4_PDLP+STSN_4_timing.csv
netlib_benchmark_1.0e-4_PDLP+STSN_5_timing.csv
netlib_benchmark_1.0e-4_PDLP_1_timing.csv
netlib_benchmark_1.0e-4_PDLP_2_timing.csv
netlib_benchmark_1.0e-4_PDLP_3_timing.csv
netlib_benchmark_1.0e-4_PDLP_4_timing.csv
netlib_benchmark_1.0e-4_PDLP_5_timing.csv
netlib_benchmark_1.0e-8_PDLP+STSN_1_timing.csv
netlib_benchmark_1.0e-8_PDLP_1_timing.csv


In [29]:
# Data for x-axis in plots: 
similarity_threshold_list = []
kappa_value_list = []
lambda_value_list = []

# Data for y-axis in plots: 
total_iterations_list = []
solved_instances_list = []
total_seconds_list = []

for file in result_summary_files:
    total_iterations = 0
    total_seconds = 0
    total_KKT_passes = 0
    instance_counter = 0
    solved_instances = 0
    total_num_errors = 0
    kkt_sgm_10 = 1
    # print(file.split("/")[-1].removesuffix(".csv"))
    similarity_threshold = file.split("threshold=")[-1].removesuffix(".csv")
    kappa=file.split("kappa=")[-1].split("_")[0]
    lambda_val=file.split("lambda=")[-1].split("_")[0]
    if not is_number(kappa):
        kappa = 0 
    if not is_number(lambda_val):
        lambda_val = 0
    if not is_number(similarity_threshold):
        similarity_threshold = "1.0"
    # print("Similarity Threshold: ", similarity_threshold)
    with open(file, "r") as csv_file:
        file_reader = csv.reader(csv_file)
        
        for line in file_reader:
            instance_counter += 1
        instance_counter -= 1
  
    with open(file, "r") as csv_file:
        file_reader = csv.reader(csv_file)
        
        first_row = True
        for line in file_reader: 
            if first_row:
                first_row = False
            else:
                termination_reason = line[1].strip()
                if termination_reason == "TERMINATION_REASON_OPTIMAL":
                    solved_instances += 1
                if termination_reason == "TERMINATION_REASON_NUMERICAL_ERROR":
                    total_num_errors += 1
                if termination_reason == "TEMINATION_REASON_ITERATION_LIMIT" or termination_reason == "TERMINATION_REASON_NUMERICAL_ERROR":
                    kkt_passes = 100000
                else:
                    kkt_passes = float(line[4].strip())
                total_iterations += int(line[2].strip())
                total_seconds += float(line[3].strip())
                total_KKT_passes += kkt_passes
                
                kkt_sgm_10 *= (kkt_passes + 10)**(1/instance_counter)
    kkt_sgm_10 = (kkt_sgm_10) - 10

    print(file.split("/")[-1])    
    print(" - Number of instances solved: ", solved_instances, "/", instance_counter) 
    # print(" - Numerical errors: ", total_num_errors)
    print(" - Total seconds: ", total_seconds)
    print(" - Total iterations: ", total_iterations) 
    print(" - Total KKT passes: ", total_KKT_passes)
    # print(" - SGM10 KKT passes: ", kkt_sgm_10)
    # Saving the stats for a comparison plot:
    total_seconds_list.append(total_seconds)
    similarity_threshold_list.append(float(similarity_threshold))
    kappa_value_list.append(float(kappa))
    lambda_value_list.append(float(lambda_val))
    total_iterations_list.append(int(total_iterations))
    solved_instances_list.append(solved_instances)

netlib_benchmark_1.0e-4_PDLP+STSN_1_timing.csv
 - Number of instances solved:  102 / 113
 - Total seconds:  142.74266856000006
 - Total iterations:  2153925
 - Total KKT passes:  2165584.0
netlib_benchmark_1.0e-4_PDLP+STSN_2_timing.csv
 - Number of instances solved:  102 / 113
 - Total seconds:  143.11463795400002
 - Total iterations:  2153925
 - Total KKT passes:  2165584.0
netlib_benchmark_1.0e-4_PDLP+STSN_3_timing.csv
 - Number of instances solved:  102 / 113
 - Total seconds:  143.10638870400004
 - Total iterations:  2153925
 - Total KKT passes:  2165584.0
netlib_benchmark_1.0e-4_PDLP+STSN_4_timing.csv
 - Number of instances solved:  102 / 113
 - Total seconds:  142.59477503699998
 - Total iterations:  2153925
 - Total KKT passes:  2165584.0
netlib_benchmark_1.0e-4_PDLP+STSN_5_timing.csv
 - Number of instances solved:  102 / 113
 - Total seconds:  143.20082608600006
 - Total iterations:  2153925
 - Total KKT passes:  2165584.0
netlib_benchmark_1.0e-4_PDLP_1_timing.csv
 - Number of 

In [28]:
# Take the average of the times for the different runs: 

for file in result_summary_files:
    print(file.split("/")[-1].removesuffix(".csv"))

average_pdlp_stsn = 0
average_pdlp = 0
for time in total_seconds_list[:5]:
    average_pdlp_stsn += time
average_pdlp_stsn /= 5

for time in total_seconds_list[5:10]:
    average_pdlp += time
average_pdlp /= 5

print("Average times: ")
print("- PDLP: ", average_pdlp)
print("- PDLP+STSN", average_pdlp_stsn)

netlib_benchmark_1.0e-4_PDLP+STSN_1_timing
netlib_benchmark_1.0e-4_PDLP+STSN_2_timing
netlib_benchmark_1.0e-4_PDLP+STSN_3_timing
netlib_benchmark_1.0e-4_PDLP+STSN_4_timing
netlib_benchmark_1.0e-4_PDLP+STSN_5_timing
netlib_benchmark_1.0e-4_PDLP_1_timing
netlib_benchmark_1.0e-4_PDLP_2_timing
netlib_benchmark_1.0e-4_PDLP_3_timing
netlib_benchmark_1.0e-4_PDLP_4_timing
netlib_benchmark_1.0e-4_PDLP_5_timing
netlib_benchmark_1.0e-8_PDLP+STSN_1_timing
netlib_benchmark_1.0e-8_PDLP_1_timing
Average times: 
- PDLP:  138.5236290098
- PDLP+STSN 142.95185926820005
